In [1]:
import torch 
from torch import nn,optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm
import re 
import collections
import itertools

# 補助関数の作成

In [2]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
    #不要な文字を削除
    text = remove_marks_regex.sub("", text)
    #?!.と単語の間に空白を挿入
    text = shift_marks_regex.sub(r"\1", text)
    return text

def parse_line(line):
    line = normalize(line.strip())
    #翻訳元と翻訳先それぞれのトークンリストを作成する
    src,trg = line.split("\t")[:2]
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens

def build_vocab(tokens):
    #ファイル中のすべての文章でのトークン数を数える
    counts = collections.Counter(tokens)
    #トークンの出現数の多い順に並べる
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    #3つのタグを追加して正引きリストと逆引き用辞書を作る
    word_list = ["<UNK>", "<SOS>", "<EOS>"] \
    + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    #末尾に終了タグをつける
    words = words + ["<EOS>"]
    #辞書を利用して数値のリストに変換する
    words = [word_dict.get(w,0) for w in words]
    seq_len = len(words)
    #長さがmax_len以下の場合はパディングする
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    #Tensorに変換して返す
    return torch.tensor(words, dtype=torch.int64), seq_len

# TranslationPairDatasetクラスの作成

In [3]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        #単語数が多い文書をフィルタリングする関数
        def filter_pair(p):
            return not(len(p[0]) > max_len or len(p[1]) > max_len)
        #ファイルを開き、パース/フィルタリングする
        with open(path)as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        #文書のペアをソースとターゲットに分ける
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        #それぞれの語彙集を作成する
        self.src_word_list, self.src_word_dict = build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = build_vocab(itertools.chain.from_iterable(trg))
        #語彙集を使用してTensorに変換する
        self.src_data = [words2tensor(words, self.src_word_dict, max_len) for words in src]
        self.trg_data = [words2tensor(words, self.trg_word_dict, max_len, -100) for words in trg]
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src, Isrc = self.src_data[idx]
        trg, Itrg = self.trg_data[idx]
        return src, Isrc, trg, Itrg

# Encoderの作成

In [4]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=10, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
    def forward(self, x, h0=None,l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h

# Decoderの作成

In [5]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x =nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h

# Translate

In [6]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    #Encoderで使用するので入力の長さもリストにしておく
    seq_len = [seq_len]
    #開始トークン準備
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    #入力文字配列をEncoderに入れてコンテキストを得る
    ctx = enc(input_tensor, l=seq_len)
    #開始トークンとコンテキストをDecoderの初期値にセット
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        #Decoderで次の単語を予測
        o,h = dec(z.view(1,1), h)
        #線形層の出力が最も大きい場所が次の単語のID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        #次の入力は今回の出力のIDを使用する
        z = wi
    #記録しておいた出力のIDを文字列に変換
    return " ".join(ds.trg_word_list[i] for i in results)

In [7]:
batch_size = 32
max_len = 39
path = "tmp.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loder = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)


## 単語数確認

In [8]:
print(len(ds.src_word_list))
print(len(ds.trg_word_list))

2177
2200


## 動作確認

In [9]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("なぜ 来る の", enc, dec)

'サポート 早く 早く 早く 早く 早く 大滝村 早く 早く 早く 早く 早く 終わり 早く 早く'

## モデルの学習

In [10]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.003)
opt_dec = optim.Adam(dec.parameters(), 0.003)
loss_f =  nn.CrossEntropyLoss()

## モデルの学習部分作成

In [11]:
from statistics import mean

def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)

for epoch in range(200):
    enc.train(), dec.train()
    losses = []
    
    for x, lx, y, ly in tqdm.tqdm(loder):
        
        #xのPackedSequenceを作るために翻訳元の長さで降順にソート
        lx, sort_idx = lx.sort(descending = True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        #翻訳元をEncoderに入れてコンテキストを得る
        ctx = enc(x, l=lx)
    
        #yのpackedSequenceを作るために翻訳先の長さで降順にソート
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        #Decoderの初期値をセット
        h0 = (ctx[0][:,sort_idx,:],ctx[1][:,sort_idx,:])
        z = y[:,:-1].detach()
        #-100のままだとEmbeddingの計算でエラーが出るので値を0にしておく
        z[z==-100] = 0
        #Decoderに通して損失関数を計算
        o, _ = dec(z, h0,l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:,1:max(ly)]).squeeze())
        #誤差逆伝播
        enc.zero_grad()
        dec.zero_grad()
        loss.backward()
        opt_enc.step()
        opt_dec.step()
        losses.append(loss.item())
       
    enc.eval()
    dec.eval()
    print(epoch, mean(losses))
    with torch.no_grad():
        print(translate("2 つ の システム 間 の 唯一 の 違い は 、 支払い の タイミング でし た 。", enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("3 百 万 人 を 超える アメリカ 人 は 、 生涯 にわたって パニック 障害 を 経験 する 。", enc, dec, max_len=max_len, device="cuda:0"))

torch.save(enc.state_dict(), './enc.pth')
torch.save(dec.state_dict(), './dec.pth')

 38%|███▊      | 8/21 [00:00<00:00, 77.54it/s]

0 6.912074157169887
は は は は の の の の の 。 。 。 。
は は は は は の の の の 。 。 。 。


 38%|███▊      | 8/21 [00:00<00:00, 79.45it/s]

1 6.265850725628081
は は は の の の の の の の の の の 。
は は は の の の の の の の の の の 。


 43%|████▎     | 9/21 [00:00<00:00, 80.94it/s]

...

197 0.0349183100320044
の うち の 違い は 、 システム 上 の アクティブ な タスク の プログラム 。
の センサス に よれ ば 、 15歳 以上 の 国民 の 識字率 は 80% で ある もの です 。


 38%|███▊      | 8/21 [00:00<00:00, 79.15it/s]

198 0.03128533757158688
の 一部 は 、 ハー バード 大学 と バージニア 大学 でし た 。
の 名 は 、 連邦 政府 が 執務 不能 の 際 に これ を 減らす こと です 。


100%|██████████| 21/21 [00:00<00:00, 80.15it/s]

199 0.028690320483985402
の うち の 違い は 、 システム 上 の アクティブ な チャネル の 1 つ です 。
の 名 は 、 その 功績 を 称え て 、 小惑星 や 月 の クレーター に 付け られ て い た 。


In [14]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("1977 年 に 彼 は ロイヤル・リヴァプール・フィルハーモニー 管弦楽 団 の アシスタント コンダクター と なり まし た 。", enc, dec)

'統計 映画 映画 映画 多数 協定 映画 映画 映画 映画 映画 プラハ 映画 映画 映画'

In [12]:
import pandas as pd
tsv = pd.read_csv('tmup.tsv', delimiter='\t')
with open("tmp.txt", "w") as f:
    for i in tsv[["sentence_A_ja", "sentence_B_ja"]].values:
        f.write(i[0] + "\t" + i[1] + "\n")
#         f.write(i[0].replace(" ", "") + "\t" + i[1].replace(" ", "") + "\n")